In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install rank-bm25

In [ ]:
!pip install kaggle --quiet

!kaggle datasets download -d paultimothymooney/recipenlg

!unzip -q recipenlg.zip -d recipenlg

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/recipenlg
License(s): other
100% 640M/643M [00:03<00:00, 105MB/s] 
100% 643M/643M [00:03<00:00, 188MB/s]


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import wordnet
import os
import ast
import pandas as pd
import re
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
# reading the dataset
df = pd.read_csv("recipenlg/RecipeNLG_dataset.csv")
df = df.dropna().reset_index(drop=True)
print(df.shape)
print(df.head())

(2231141, 7)
   Unnamed: 0                  title  \
0           0    No-Bake Nut Cookies   
1           1  Jewell Ball'S Chicken   
2           2            Creamy Corn   
3           3          Chicken Funny   
4           4   Reeses Cups(Candy)     

                                         ingredients  \
0  ["1 c. firmly packed brown sugar", "1/2 c. eva...   
1  ["1 small jar chipped beef, cut up", "4 boned ...   
2  ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg...   
3  ["1 large whole chicken", "2 (10 1/2 oz.) cans...   
4  ["1 c. peanut butter", "3/4 c. graham cracker ...   

                                          directions  \
0  ["In a heavy 2-quart saucepan, mix brown sugar...   
1  ["Place chipped beef on bottom of baking dish....   
2  ["In a slow cooker, combine all ingredients. C...   
3  ["Boil and debone chicken.", "Put bite size pi...   
4  ["Combine first four ingredients and press in ...   

                                              link    source  \
0   www.

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1, 2))
train_vectors = vectorizer.fit_transform(train_df['ingredients'])

nn = NearestNeighbors(n_neighbors=1, metric="cosine")
nn.fit(train_vectors)


NearestNeighbors(metric='cosine', n_neighbors=10)

In [ ]:
def find_recipe(user_input):
    input_ingredients = extract_ingredients(user_input)

    input_vector = vectorizer.transform([input_ingredients])
    distances, indices = nn.kneighbors(input_vector)

    best_recipe = train_df.iloc[indices[0][0]]
    return best_recipe['title'], best_recipe['ingredients'], best_recipe['directions']


print(find_recipe("flour, apple"))

('Caramel Apple', 'jigger apple pucker jigger schnapps butterscotch', '["Pour apple Pucker into shot glass first and then top with butterscotch Schnapps."]')
